###### U20EC147
# Experiment 6
### Aim : Design a SVC with different kernel functions, and train it on diabetes dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn import svm

In [ ]:
REPLACEMENT_SCHEME = 'category_specific_mean'

In [ ]:
df = pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [ ]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
zero_not_allowed = ["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]

for column in zero_not_allowed:
    df[column] = df[column].replace(0, np.NaN)
    if REPLACEMENT_SCHEME == 'mean':
        replacement_value = int(df[column].mean(skipna=True))
        df[column] = df[column].replace(np.NaN, replacement_value)
    elif REPLACEMENT_SCHEME == 'median':
        replacement_value = int(df[column].median(skipna=True))
        df[column] = df[column].replace(np.NaN, replacement_value)
    elif REPLACEMENT_SCHEME == 'category_specific_mean':
        one_replacement_value = int(df["Insulin"][df["Outcome"] == 1].mean(skipna=True))
        zero_replacement_value = int(df["Insulin"][df["Outcome"] == 0].mean(skipna=True))
        df.loc[df["Outcome"] == 1, column] = one_replacement_value
        df.loc[df["Outcome"] == 0, column] = zero_replacement_value
    elif REPLACEMENT_SCHEME == 'category_specific_median':
        one_replacement_value = int(df["Insulin"][df["Outcome"] == 1].median(skipna=True))
        zero_replacement_value = int(df["Insulin"][df["Outcome"] == 0].median(skipna=True))
        df.loc[df["Outcome"] == 1, column] = one_replacement_value
        df.loc[df["Outcome"] == 0, column] = zero_replacement_value

In [ ]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,79.166667,79.166667,79.166667,156.520833,156.520833,0.471876,33.240885,0.348958
std,3.369578,15.262444,15.262444,15.262444,36.248305,36.248305,0.331329,11.760232,0.476951
min,0.000000,68.000000,68.000000,68.000000,130.000000,130.000000,0.078000,21.000000,0.000000
25%,1.000000,68.000000,68.000000,68.000000,130.000000,130.000000,0.243750,24.000000,0.000000
50%,3.000000,68.000000,68.000000,68.000000,130.000000,130.000000,0.372500,29.000000,0.000000
75%,6.000000,100.000000,100.000000,100.000000,206.000000,206.000000,0.626250,41.000000,1.000000
max,17.000000,100.000000,100.000000,100.000000,206.000000,206.000000,2.420000,81.000000,1.000000


In [ ]:
df['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [ ]:
df.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,68.0,68.0,68.0,130.0,130.0,0.429734,31.190000
1,4.865672,100.0,100.0,100.0,206.0,206.0,0.550500,37.067164


In [ ]:
x = df.iloc[:,:-2]
y = df.iloc[:,-1]

In [ ]:
x

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction
0,6,100.0,100.0,100.0,206.0,206.0,0.627
1,1,68.0,68.0,68.0,130.0,130.0,0.351
2,8,100.0,100.0,100.0,206.0,206.0,0.672
3,1,68.0,68.0,68.0,130.0,130.0,0.167
4,0,100.0,100.0,100.0,206.0,206.0,2.288
...,...,...,...,...,...,...,...
763,10,68.0,68.0,68.0,130.0,130.0,0.171
764,2,68.0,68.0,68.0,130.0,130.0,0.340
765,5,68.0,68.0,68.0,130.0,130.0,0.245
766,1,100.0,100.0,100.0,206.0,206.0,0.349


In [ ]:
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)
x

array([[ 0.63994726,  1.36589591,  1.36589591, ...,  1.36589591,
         1.36589591,  0.46849198],
       [-0.84488505, -0.73212021, -0.73212021, ..., -0.73212021,
        -0.73212021, -0.36506078],
       [ 1.23388019,  1.36589591,  1.36589591, ...,  1.36589591,
         1.36589591,  0.60439732],
       ...,
       [ 0.3429808 , -0.73212021, -0.73212021, ..., -0.73212021,
        -0.73212021, -0.68519336],
       [-0.84488505,  1.36589591,  1.36589591, ...,  1.36589591,
         1.36589591, -0.37110101],
       [-0.84488505, -0.73212021, -0.73212021, ..., -0.73212021,
        -0.73212021, -0.47378505]])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [ ]:
model1 = svm.SVC(kernel = 'linear')
model2 = svm.SVC(kernel = 'poly')
model3 = svm.SVC(kernel = 'rbf')
model4 = svm.SVC(kernel = 'sigmoid')

In [ ]:
model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)
model4.fit(x_train,y_train)

SVC(kernel='sigmoid')

In [ ]:
y_pred1 = model1.predict(x_test)
y_pred2 = model2.predict(x_test)
y_pred3 = model3.predict(x_test)
y_pred4 = model4.predict(x_test)

In [ ]:
print("test_data_accuracy_linear: ", accuracy_score(y_test, y_pred1))
print("test_data_accuracy_poly: ", accuracy_score(y_test, y_pred2))
print("test_data_accuracy_rbf: ", accuracy_score(y_test, y_pred3))
print("test_data_accuracy_sigmoid: ", accuracy_score(y_test, y_pred4))

test_data_accuracy_linear:  1.0
test_data_accuracy_poly:  1.0
test_data_accuracy_rbf:  1.0
test_data_accuracy_sigmoid:  0.9935064935064936


In [ ]:
y_pred1 = model1.predict(x_train)
y_pred2 = model2.predict(x_train)
y_pred3 = model3.predict(x_train)
y_pred4 = model4.predict(x_train)

In [ ]:
print("train_data_accuracy_linear: ", accuracy_score(y_train, y_pred1))
print("train_data_accuracy_poly: ", accuracy_score(y_train, y_pred2))
print("train_data_accuracy_rbf: ", accuracy_score(y_train, y_pred3))
print("train_data_accuracy_sigmoid: ", accuracy_score(y_train, y_pred4))

train_data_accuracy_linear:  1.0
train_data_accuracy_poly:  1.0
train_data_accuracy_rbf:  1.0
train_data_accuracy_sigmoid:  0.995114006514658
